# Tune Model Hyperparameters


You may have noticed that AutoGluon tries multiple models from decision trees to neural network during fit. If you are familiar with these models, you can specify what models AutoGluon should try, and model hyperparameters AutoGluon should search. 

Let's first figure what models AutoGluon trained, with their performances on the test set through the {class}`autogluon.tabular.TabularPredictor.leaderboard` method.


In [1]:
#@title Install autogluon
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 14.8 MB/s 
     |████████████████████████████████| 203 kB 47.1 MB/s 
     |████████████████████████████████| 48 kB 6.4 MB/s 
     |████████████████████████████████| 272 kB 65.3 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
     |████████████████████████████████| 61 kB 245 kB/s 
     |████████████████████████████████| 132 kB 52.7 MB/s 
     |████████████████████████████████| 802 kB 56.1 MB/s 
     |████████████████████████████████| 38.1 MB 48.9 MB/s 
     |████████████████████████████████| 1.0 MB 49.0 MB/s 
     |████████████████████████████████| 54.5 MB 104 kB/s 
     |████████████████████████████████| 1.6 MB 50.5 MB/s 
     |████████████████████████████████| 281 kB 55.5 MB/s 
     |████████████████████████████████| 13.5 MB 50.5 MB/s 
     |███████████████████████████████

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

url = 'https://autogluon.s3.amazonaws.com/datasets/Inc/'
train_data = TabularDataset(url+'train.csv')
test_data = TabularDataset(url+'test.csv')

predictor = TabularPredictor(label='class').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20220708_204548/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220708_204548/"
AutoGluon Version:  0.5.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify 

In [4]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.8912,0.560394,15.908415,0.005856,1.393187,2,True,14
1,XGBoost,0.8872,0.033904,2.089937,0.033904,2.089937,1,True,11
2,LightGBMLarge,0.8856,0.075173,2.833156,0.075173,2.833156,1,True,13
3,CatBoost,0.8836,0.027265,34.507984,0.027265,34.507984,1,True,7
4,LightGBM,0.8824,0.149639,3.550111,0.149639,3.550111,1,True,4
5,LightGBMXT,0.8792,0.167164,5.897917,0.167164,5.897917,1,True,3
6,NeuralNetFastAI,0.8640,0.068323,49.774255,0.068323,49.774255,1,True,10
7,RandomForestEntr,0.8620,0.315026,12.587335,0.315026,12.587335,1,True,6
8,RandomForestGini,0.8588,0.312141,15.366402,0.312141,15.366402,1,True,5
9,NeuralNetTorch,0.8584,0.060468,49.021325,0.060468,49.021325,1,True,12


Each row contains a model, with its validation score (`score_val`), prediction time on the validation set (`pred_time_val`), and training time (`fit_time`). The score is normalized from the validation metric (see {doc}`evaluations` for more details), the higher the better. 

From this leaderboard, you can get a sense about each model's accuracy versus computational cost on your data. For example, the model with highest score is `WeightedEnsemble_L2`, which a weighted combination of multiple models. `XGBoost` performs well as long as other tree models. Neural networks such as `NeuralNetFastAI` and `NeuralNetTorch` report medium scores but are more expensive to train.

```{hint}
Beyound on the validation set, you can evaluate models on a test dataset by `predictor.leaderboard(test_data, ...)`. But if you are doing it, you may use `test_data` as validation data to tune your models.
```

Next let's manually construct the search space. Through the `hyperparameters` argument, we specify the space for each type of the model. For each type, we can either specify a hyperparameter to a particular value, or a set of candidates. Then we specify how to search a good hyperparameter configuration through the `hyperparameter_tune_kwargs` argument.



In [9]:
from autogluon.core import space

nn_options = { # for neural network
  'num_epochs': 10, # controls the training time of neural networks
  # search learning rate on log-scale
  'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
  # activation functions, the first entry is the default
  'activation': space.Categorical('relu', 'softrelu', 'tanh'),
  # dropout probability
  'dropout_prob': space.Real(0.0, 0.5, default=0.1)
}

gbm_options = {  # for lightGBM
  # controls the training time of GBM models
  'num_boost_round': 100,  
  # number of leaves, controls the model complexity 
  'num_leaves': space.Int(lower=26, upper=66, default=36), 
}

# Only try two models
hyperparameters = {'GBM': gbm_options, 'NN_TORCH': nn_options }

# How to search in the hyperparameter space.
hyperparameter_tune_kwargs = {
    # Try at most 5 hyperparameter configurations for each type of model
    'num_trials': 5,
    # Search on local machine, you can search on a cluster
    'scheduler': 'local',
    # Automatically choose the hyperparameter optimization method
    'searcher': 'auto'
}

predictor = TabularPredictor(label='class').fit(
    train_data, hyperparameters=hyperparameters, 
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20220708_213359/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220708_213359/"
AutoGluon Version:  0.5.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify 

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	0.8756	 = Validation score   (accuracy)
	1.87s	 = Training   runtime
	0.07s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.8764	 = Validation score   (accuracy)
	0.98s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.8756	 = Validation score   (accuracy)
	0.93s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.8186	 = Validation score   (accuracy)
	0.8s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.8742	 = Validation score   (accuracy)
	0.92s	 = Training   runtime
	0.04s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: NeuralNetTorch/T1 ...
	0.8536	 = Validation score   (accuracy)
	11.72s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: NeuralNetTorch/T2 ...
	0.859	 = Validation score   (accuracy)
	24.72s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: NeuralNetTorch/T3 ...
	0.8442	 = Validation score   (accuracy)
	25.28s	 = Training   runtime
	0.2s	 = Validation runtime
Fitted model: NeuralNetTorch/T4 ...
	0.8536	 = Validation score   (accuracy)
	48.76s	 = Training   runtime
	0.19s	 = Validation runtime
Fitted model: NeuralNetTorch/T5 ...
	0.8504	 = Validation score   (accuracy)
	58.06s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8784	 = Validation score   (accuracy)
	1.92s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 180.58s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20

Use `leaderboard` again, you will see the model performance of each trial.  

In [10]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.8784,0.769726,125.481239,0.009162,1.917079,2,True,11
1,LightGBM/T2,0.8764,0.031383,0.979046,0.031383,0.979046,1,True,2
2,LightGBM/T3,0.8756,0.048398,0.927761,0.048398,0.927761,1,True,3
3,LightGBM/T1,0.8756,0.067719,1.871549,0.067719,1.871549,1,True,1
4,LightGBM/T5,0.8742,0.037126,0.919814,0.037126,0.919814,1,True,5
5,NeuralNetTorch/T2,0.8590,0.110289,24.722598,0.110289,24.722598,1,True,7
6,NeuralNetTorch/T1,0.8536,0.095418,11.724735,0.095418,11.724735,1,True,6
7,NeuralNetTorch/T4,0.8536,0.193634,48.760532,0.193634,48.760532,1,True,9
8,NeuralNetTorch/T5,0.8504,0.208774,58.057219,0.208774,58.057219,1,True,10
9,NeuralNetTorch/T3,0.8442,0.198582,25.281251,0.198582,25.281251,1,True,8


Note that constructing a good search space requires domain knowledge. The space should be large enough to cover good configurations, but should not be too large to search. You can start with AutoGluon's default hyperparameters, and interatively refine the search space.


```{seealso}
Know all model type (LINK) and their hyperparameters (LINK). 

Know all search algorithms (LINK)

You can also add your customized models.

External docs about HPO.
```

(TODO, how to reuse models traind before when you are iteratively refining the search space).